In [26]:
!pip install PyPDF2 -q
!pip install fitz -q
!pip install PyMuPDF -q
!pip install fitz -q
!pip install reportlab -q
!pip install pymorphy2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [27]:
import collections
import io
import random

from PyPDF2 import PdfWriter, PdfReader
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfgen import canvas

import PyPDF2
import glob
import fitz
import json
import re

import pymorphy2

In [6]:
def find_files(path_to_folder: str = '/content'):
    pdf_files = glob.glob(path_to_folder + '/*.pdf')
    #pdf_files = glob.glob(folder_path + '/**/*.pdf', recursive=True) для рекурсивного поиска по сабдиректориям
    return pdf_files

In [18]:
def find_labels(pdf_file_):
    page_num = 1
    labels = {}
    doc = fitz.open(pdf_file_)
    page = doc.load_page

    for page in doc:
        page_labels = []
        for annot in page.annots():
            text = page.get_textbox(annot.rect)
            page_labels.append(text)
        labels[page_num] = page_labels
        page_num += 1
    return labels



def pdf2text(pdf_files):
    data = {}
    for pdf_file_ in pdf_files:
        file_map = {}

        labels = find_labels(pdf_file_)

        pdf_file = open(pdf_file_, 'rb')
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        for page in range(num_pages):
            pdf_page = pdf_reader.pages[page]
            pdf_text = pdf_page.extract_text()

            file_map[page + 1] = {'text': pdf_text, 'labels': labels[page + 1]}
        data[pdf_file_[pdf_file_.rfind('/') + 1:]] = file_map

        pdf_file.close()

    return data

In [258]:
def listToString(s):

    str1 = " "

    for ele in s:
        str1 += ele + " "

    return str1


def cleanText(text):                            #очистка текста
    text = text.lower()

    text = re.sub('(-)', '', text)

    text = re.sub('\n', '', text)

    text = re.sub(r'[_\n/_–<\*+«,\#+\№\"\-+\_+\=+{\?+\»%!+\&\}^\+\;\+\>«+"\(\)\/+\:\\+.]', r' ', text)

    text = re.sub(r'[abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ]', r' ', text)

    text = re.sub(r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)', r' ', text)

    text = re.sub(r'(\d+\s\d+)|(\d+)',' ', text)

    text = re.sub(r'\s+', ' ', text)

    text = re.sub(r'\uf06c\uf020', ' ', text)

    return text

def morphed(text):                             #лемматизация
    text = cleanText(text)
    #text = text.lower()
    splited = text.split(' ')
    for i in range(len(splited)):
        splited[i] = morph.parse(splited[i])[0].normal_form
    text = listToString(splited)
    return text

In [227]:
def get_mask_(text, set_ethelone):
    mask = []
    for word in text.split(' '):
        if word in set_ethelone:
            mask.append(1)
        elif len(word) < 3:
            mask.append(-1)
        else:
            mask.append(0)
    return mask

def get_mask(data, set_ethelone, ethelone_size):
    mask_map = {}
    for fil in data.keys():
        file_masks = {}
        for page_num in data[fil].keys():
            text = data[fil][page_num]['text']
            text = morphed(text)
            mask = get_mask_(text, set_ethelone)
            mask = process_mask(mask, ethelone_size)
            file_masks[page_num] = mask
        mask_map[fil] = file_masks
    return mask_map

In [253]:
def process_mask(row_mask, ethelone_size, t=5):
    '''
    Убирает все последовательности единиц короче <t>
    Если есть один или два нуля между единицами, кастует его к единице
    '''
    n = len(row_mask)
    post_mask = [0]*(n)
    hold = t # длина условного эталона - 5
    c=0

    is_between_ones = False
    second_flag = True
    p_start = 0
    for position, i in enumerate(row_mask):
        if i == 1:
            is_between_ones = True
        if i == 0:
            is_between_ones = False
            second_flag = True
        if i == -1 and is_between_ones:
            if second_flag:
                p_start = position - 1
                second_flag = False
        if i == 1 and is_between_ones and not second_flag:
            for j in range(p_start, position):
                row_mask[j] = 1
            is_between_ones = False
            second_flag = True

    for i in range(len(row_mask)):
        if row_mask[i] == -1:
            row_mask[i] = 0

    for i in range(n):
        if row_mask[i]:
            c+=1
        else:
            if c >= hold:
                for j in range(i,i-c,-1): post_mask[j]=1
                c = 0


    for i in range(1,n-2):
        if post_mask[i]==0 and post_mask[i-1]==1 and post_mask[i+1]==1:
            post_mask[i]=1
        if i >= 2 and post_mask[i]==0 and  post_mask[i+1]==0 and post_mask[i-1]==1 and post_mask[i+2]==1:
            post_mask[i]=1

    post_mask = filter_sequence(post_mask, threshold = ethelone_size*0.2)

    return post_mask

In [241]:
def filter_sequence(sequence, threshold=5):
    filtered_sequence = sequence.copy()
    flag = True
    start_pos = 0
    current_streak = 0
    for i in range(len(sequence)):
        if sequence[i] == 1:
            if flag:
                start_pos = i
                flag = False
            current_streak += 1
        else:
            current_streak -= 2
            if current_streak <= 0 and current_streak < threshold and not flag:
                for j in range(start_pos, i):
                    filtered_sequence[j] = 0
            current_streak = 0
            flag = True

    return filtered_sequence

In [231]:
def normalize_pdf(input_path, output_path, text_to_highlight):
    words_count = highlight_pdf(input_path, output_path, text_to_highlight)
    add_title_pdf(output_path, output_path, words_count)


def highlight_pdf(input_path, output_path, text_to_highlight):
    doc = fitz.open(input_path)
    words_count = collections.defaultdict(int)
    for i in range(len(doc)):
        cur_page_mask = text_to_highlight[i + 1]
        page = doc[i]
        words = page.get_text("words")
        for j in range(len(words)):
            word = words[j]
            if j - 1 < len(cur_page_mask) and cur_page_mask[j - 1]:
                rect = fitz.Rect(word[:4])
                highlight = page.add_highlight_annot(rect)

                words_count[i] += 1

    doc.save(output_path)
    doc.close()
    return words_count


def add_title_pdf(input_path, output_path, words_count):
    font_path = 'times_new_roman.ttf'
    font_size = 8
    title_header = 'Highlights'
    header_font_size = 18
    columns_count = 5

    header_top_padding = 40
    header_start_padding = 220
    link_top_padding = 100
    link_start_padding = 50

    height = 792
    width = 612

    title_page = {
        f'Страница {key + 1}({words_count[key]})': key + 1 for key in sorted(words_count)
    }

    output_pdf = PdfWriter()

    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)

    pdfmetrics.registerFont(TTFont('CustomFont', font_path))

    if title_header:
        can.setFont("CustomFont", header_font_size)
        can.drawString(header_start_padding, height - header_top_padding, title_header)

    can.setFont("CustomFont", font_size)

    y_position = height - link_top_padding
    y_delta = y_position // len(title_page)
    x_delta = (width - 50) // columns_count
    for i, (title, page_number) in enumerate(title_page.items()):
        x = link_start_padding + x_delta * (i % columns_count)
        y = y_position + y_delta * (i % columns_count)
        can.drawString(x, y, title.encode('utf8', 'ignore'))
        can.linkURL(f"#page={page_number + 1}", (x, y, x + x_delta, y + 15), relative=0)
        y_position -= y_delta

    can.save()

    packet.seek(0)
    new_pdf = PdfReader(packet)

    input_pdf = PdfReader(open(input_path, 'rb'))

    output_pdf.add_page(new_pdf.pages[0])
    for i in range(len(input_pdf.pages)):
        output_pdf.add_page(input_pdf.pages[i])

    with open(output_path, 'wb') as f:
        output_pdf.write(f)

In [276]:
def find_theme(text: str):
    text = text.replace('\n', '')
    text = text.replace('-', '')
    matches_ = re.findall(r'«([^«»]*)»', text)
    matches_for_replace = re.findall(r'«[^«»]*»', text)
    for i, m in enumerate(matches_for_replace):
        text = text.replace(m, matches_[i])
    matches = re.findall(r'«(.+?)»', text)
    if len(matches) == 0:
        matches = matches_
    len_counter = 0
    chosed_text = ''
    for text in matches:
        if len(text) > len_counter:
            len_counter = len(text)
            chosed_text = text
    return chosed_text

In [280]:
input_pdf_path = "input.pdf"
output_pdf_path = "output.pdf"

#pdf_files = find_files() нужна, если input_pdf не один

data = pdf2text([input_pdf_path])

title_text = data[input_pdf_path[input_pdf_path.rfind('/') + 1 : ]][1]['text']

ethelone = find_theme(title_text)

morph = pymorphy2.MorphAnalyzer()

ethelone_morphed = morphed(ethelone)
ethelone_size = len(ethelone_morphed.split(' '))

set_ethelone_morphed = set([word for word in ethelone_morphed.split(' ') if len(word) >= 3])

masks = get_mask(data, set_ethelone_morphed, ethelone_size)

normalize_pdf(input_pdf_path, output_pdf_path, masks[input_pdf_path])